In [1]:
!pip install pyngrok fastapi uvicorn nest_asyncio
!pip install py-AutoClean
!pip install pycaret
!pip install pycaret py-AutoClean pandas matplotlib seaborn
!pip install python-multipart
!pip install httpx
import httpx
from pyngrok import ngrok
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import nest_asyncio
import pandas as pd
from AutoClean import AutoClean
import matplotlib.pyplot as plt
import seaborn as sns
import io
from fastapi.responses import StreamingResponse, JSONResponse
import base64
from typing import List, Dict
from pycaret.anomaly import *
from pycaret.classification import setup, compare_models
from textwrap import wrap
import traceback
from pydantic import BaseModel



# Apply nest_asyncio
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()

# CORS middleware configuration
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

def plot_to_base64(plt):
    """Convert matplotlib plot to base64 string"""
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight')
    buf.seek(0)
    image_base64 = base64.b64encode(buf.getvalue()).decode('utf-8')
    plt.close()
    return image_base64

def generate_dynamic_insights(df, column):
    """
    Generates automated business insights based on unique value distributions and category dominance.
    """
    unique_values = df[column].nunique()
    total_values = len(df[column])

    if unique_values > total_values * 0.5:
        return (f"{column} appears to contain unique identifiers (e.g., Order IDs, Customer IDs).\n"
                " Actionable Insight: These columns are not useful for direct aggregation. Instead, analyze:\n"
                "   - Customer retention, repeat purchases, and lifetime value.\n"
                "   - Unique transaction counts and fraud detection opportunities.")

    top_category_percentage = df[column].value_counts(normalize=True).iloc[0]
    if top_category_percentage > 0.5:
        top_category = df[column].value_counts().idxmax()
        return (f" {column} is dominated by one category ({top_category}, {top_category_percentage:.1%}).\n"
                " Actionable Insight: Investigate why this category is so prevalent:\n"
                "   - Is it due to business focus, customer preference, or a data issue?\n"
                "   - Consider diversifying offerings or marketing strategies.")

    elif unique_values > 50 and top_category_percentage < 0.05:
        return (f" {column} contains a large number of unique categories with no clear dominance.\n"
                " Actionable Insight:\n"
                "   - Use clustering techniques (e.g., K-Means, PCA) to group similar categories.\n"
                "   - Find hidden patterns in customer preferences or product variations.")

    elif unique_values > 5 and top_category_percentage < 0.3:
        return (f" {column} has a well-balanced category distribution.\n"
                " Actionable Insight:\n"
                "   - Identify emerging trends within the data.\n"
                "   - Explore potential correlations between different categories and key performance metrics.")

    rare_categories = (df[column].value_counts(normalize=True) < 0.01).sum()
    if rare_categories > unique_values * 0.5:
        return (f" {column} contains many low-frequency categories (potential outliers or niche segments).\n"
                " Actionable Insight:\n"
                "   - Investigate if these rare categories represent special cases, premium products, or errors.\n"
                "   - Consider whether to merge, remove, or highlight these categories in reporting.")

    elif unique_values < total_values * 0.05:
        return (f" {column} likely represents key business attributes (e.g., products, locations, customer types).\n"
                " Actionable Insight:\n"
                "   - Optimize inventory management and marketing efforts based on category trends.\n"
                "   - Identify best-selling products, popular regions, and top-performing customer segments.")

    return (f" {column} contains valuable categorical data that could provide business insights.\n"
            " Actionable Insight:\n"
            "   - Analyze distribution trends to understand seasonality, regional preferences, or behavioral shifts.\n"
            "   - Look for correlations with key business metrics like revenue, sales, or retention rates.")

def generate_summary_report_image(df):
    """
    Generates a structured business insights report including statistical summaries and actionable insights.
    """
    insights = []

    # Get numerical columns excluding datetime columns
    numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns
    datetime_columns = df.select_dtypes(include=['datetime64']).columns

    # Title Section
    insights.append(("BUSINESS INSIGHTS REPORT\n", "black", "bold"))
    insights.append(("This report provides key trends and statistics from the dataset with actionable insights.", "black", "regular"))

    # Add Numerical Summary
    if len(numerical_columns) > 0:
        insights.append(("\nNUMERICAL DATA SUMMARY\n", "black", "bold"))
        numerical_summary = df[numerical_columns].describe().transpose()

        for column in numerical_summary.index:
            mean_value = numerical_summary.loc[column, 'mean']
            min_value = numerical_summary.loc[column, 'min']
            max_value = numerical_summary.loc[column, 'max']
            std_dev = numerical_summary.loc[column, 'std']

            insights.append((f"- {column}: Mean = {mean_value:.2f}, Min = {min_value:.2f}, Max = {max_value:.2f}, Std Dev = {std_dev:.2f}", "black", "regular"))

            if std_dev > mean_value * 0.5:
                insights.append(("   High variability detected! Consider investigating fluctuations.", "red", "regular"))
            elif std_dev < mean_value * 0.1:
                insights.append(("   Stable trend observed. Predictable behavior can be leveraged.", "green", "regular"))

    # Add DateTime Summary if present
    if len(datetime_columns) > 0:
        insights.append(("\nTIME-BASED DATA SUMMARY\n", "black", "bold"))
        for column in datetime_columns:
            min_date = df[column].min()
            max_date = df[column].max()
            date_range = max_date - min_date
            insights.append((f"- {column}:", "black", "bold"))
            insights.append((f"  Date Range: {min_date.strftime('%Y-%m-%d')} to {max_date.strftime('%Y-%m-%d')}", "black", "regular"))
            insights.append((f"  Total Time Span: {date_range.days} days", "black", "regular"))

    # Create figure
    fig, ax = plt.subplots(figsize=(12, 8), dpi=100)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_frame_on(False)

    # Display text inside the figure
    y_position = 0.95
    for text, color, weight in insights:
        wrapped_text = "\n".join(wrap(text, width=90))
        ax.text(
            0.02, y_position, wrapped_text,
            fontsize=14,
            va="top", ha="left",
            family="Times New Roman",
            fontweight=weight,
            linespacing=1.5,
            color=color
        )
        y_position -= 0.05

    return plt


# @app.post("/analyze-data")
# async def analyze_data(file: UploadFile = File(...)):
#     # Read the uploaded CSV file
#     content = await file.read()
#     data = pd.read_csv(io.StringIO(content.decode('utf-8')))

#     # Initial analysis results
#     initial_analysis = {
#         "null_values": data.isnull().sum().to_dict(),
#         "duplicates": int(data.duplicated().sum()),
#         "row_count": len(data),
#         "column_types": data.dtypes.astype(str).to_dict()
#     }

#     # Clean the data
#     cleaned_data = AutoClean(
#         data,
#         mode="auto",
#         encode_categ='auto',
#         missing_num=True,
#         missing_categ='auto',
#         outliers='delete',
#         duplicates=True,
#         extract_datetime='s'
#     )

#     # Post-cleaning analysis
#     cleaned_analysis = {
#         "null_values": cleaned_data.output.isnull().sum().to_dict(),
#         "duplicates": int(cleaned_data.output.duplicated().sum()),
#         "row_count": len(cleaned_data.output)
#     }

#     # Generate visualizations
#     images = []

#     # Box plots
#     numerical_columns = cleaned_data.output.select_dtypes(include=['int64', 'float64']).columns
#     for column in numerical_columns:
#         plt.figure(figsize=(8, 6))
#         sns.boxplot(x=cleaned_data.output[column])
#         plt.xlabel(column)
#         plt.title(f'Box Plot of {column}')
#         images.append({
#             "name": f"boxplot_{column}",
#             "image": plot_to_base64(plt)
#         })

#     # Generate categorical insights and plots
#     categorical_insights = []
#     categorical_columns = cleaned_data.output.select_dtypes(include=['object']).columns

#     for column in categorical_columns:
#         plt.figure(figsize=(10, 5))
#         top_20 = cleaned_data.output[column].value_counts().nlargest(20)
#         sns.barplot(y=top_20.index, x=top_20.values, palette="Blues_r")
#         plt.xlabel("Count")
#         plt.ylabel(column)
#         plt.title(f"Top 20 {column} Categories")

#         images.append({
#             "name": f"categorical_{column}",
#             "image": plot_to_base64(plt)
#         })

#         insight = generate_dynamic_insights(cleaned_data.output, column)
#         categorical_insights.append({
#             "column": column,
#             "insight": insight
#         })

#     # Generate summary report
#     summary_plt = generate_summary_report_image(cleaned_data.output)
#     images.append({
#         "name": "summary_report",
#         "image": plot_to_base64(summary_plt)
#     })

#     return {
#         "initial_analysis": initial_analysis,
#         "cleaned_analysis": cleaned_analysis,
#         "images": images,
#         "categorical_insights": categorical_insights
#     }

@app.exception_handler(Exception)
async def global_exception_handler(request, exc):
    return JSONResponse(
        status_code=500,
        content={
            "error": str(exc),
            "detail": traceback.format_exc()
        }
    )
class CsvRequest(BaseModel):
    cloudinary_url: str

@app.post("/analyze-data")
async def analyze_data(csv_request: CsvRequest):
    try:
        print(f"Received Cloudinary URL: {csv_request.cloudinary_url}")
        async with httpx.AsyncClient() as client:
            response = await client.get(csv_request.cloudinary_url)
            if response.status_code != 200:
                raise HTTPException(
                    status_code=400,
                    detail="Failed to download CSV from Cloudinary URL"
                )
        content = response.content
        print(f"Downloaded content length: {len(content)} bytes")

        # Try different encodings
        encodings_to_try = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']
        data = None

        for encoding in encodings_to_try:
            try:
                print(f"Trying {encoding} encoding...")
                data = pd.read_csv(io.StringIO(content.decode(encoding)))
                print(f"Successfully read CSV with {encoding} encoding")
                break
            except UnicodeDecodeError:
                continue
            except Exception as e:
                print(f"Error with {encoding}: {str(e)}")
                continue

        if data is None:
            raise HTTPException(
                status_code=400,
                detail="Could not read CSV file with any supported encoding"
            )

        # Convert date columns to datetime
        date_columns = ['Order Date', 'Ship Date']  # Add any other date columns here
        for col in date_columns:
            if col in data.columns:
                data[col] = pd.to_datetime(data[col])

        print(f"CSV shape: {data.shape}")
        print(f"CSV columns: {data.columns.tolist()}")

        # Rest of your code remains the same...
        # Initial analysis results
        initial_analysis = {
            "null_values": data.isnull().sum().to_dict(),
            "duplicates": int(data.duplicated().sum()),
            "row_count": len(data),
            "column_types": data.dtypes.astype(str).to_dict()
        }

        # Clean the data
        cleaned_data = AutoClean(
            data,
            mode="auto",
            encode_categ='auto',
            missing_num=True,
            missing_categ='auto',
            outliers='delete',
            duplicates=True,
            extract_datetime='s'
        )

        # Post-cleaning analysis
        cleaned_analysis = {
            "null_values": cleaned_data.output.isnull().sum().to_dict(),
            "duplicates": int(cleaned_data.output.duplicated().sum()),
            "row_count": len(cleaned_data.output)
        }

        # Generate visualizations
        images = []

        try:
            # Box plots
            numerical_columns = cleaned_data.output.select_dtypes(include=['int64', 'float64']).columns
            for column in numerical_columns:
                plt.figure(figsize=(8, 6))
                sns.boxplot(x=cleaned_data.output[column])
                plt.xlabel(column)
                plt.title(f'Box Plot of {column}')
                images.append({
                    "name": f"boxplot_{column}",
                    "image": plot_to_base64(plt)
                })

            # Generate categorical insights and plots
            categorical_insights = []
            categorical_columns = cleaned_data.output.select_dtypes(include=['object']).columns

            for column in categorical_columns:
                plt.figure(figsize=(10, 5))
                top_20 = cleaned_data.output[column].value_counts().nlargest(20)
                sns.barplot(y=top_20.index, x=top_20.values, palette="Blues_r")
                plt.xlabel("Count")
                plt.ylabel(column)
                plt.title(f"Top 20 {column} Categories")

                images.append({
                    "name": f"categorical_{column}",
                    "image": plot_to_base64(plt)
                })

                insight = generate_dynamic_insights(cleaned_data.output, column)
                categorical_insights.append({
                    "column": column,
                    "insight": insight
                })

            # Generate summary report
            summary_plt = generate_summary_report_image(cleaned_data.output)
            images.append({
                "name": "summary_report",
                "image": plot_to_base64(summary_plt)
            })

        except Exception as viz_error:
            print(f"Error generating visualizations: {str(viz_error)}")
            print(f"Visualization error traceback: {traceback.format_exc()}")
            raise HTTPException(
                status_code=500,
                detail={
                    "error": "Error generating visualizations",
                    "message": str(viz_error)
                }
            )

        return {
            "message": "Analysis completed successfully",
            "initial_analysis": initial_analysis,
            "cleaned_analysis": cleaned_analysis,
            "images": images,
            "categorical_insights": categorical_insights
        }
    except Exception as e:
        print(f"Error processing file: {str(e)}")
        print(f"Traceback: {traceback.format_exc()}")
        raise HTTPException(
            status_code=500,
            detail={
                "error": str(e),
                "traceback": traceback.format_exc()
            }
        )

@app.get("/")
def read_root():
    return {"message": "FastAPI Data Analysis Service is running!"}

# Setup ngrok
ngrok.set_auth_token("2swgwcEJ5hsXEst7a5WBLtv58s8_5FtZDTirtSBKrSL4e8HUR")  # Replace with your ngrok auth token
public_url = ngrok.connect(8000)
print("FastAPI is publicly accessible at:", public_url)

# Run the FastAPI app
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [19234]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


FastAPI is publicly accessible at: NgrokTunnel: "https://ad3a-34-139-120-42.ngrok-free.app" -> "http://localhost:8000"
Received Cloudinary URL: https://res.cloudinary.com/dwd6kau8a/raw/upload/v1739786984/z7cnfb9lbhre2zxujyef.csv
Downloaded content length: 2305990 bytes
Trying utf-8 encoding...
Trying latin-1 encoding...
Successfully read CSV with latin-1 encoding
CSV shape: (9994, 21)
CSV columns: ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit']
AutoClean process completed in 4.800643 seconds
Logfile saved to: /content/autoclean.log


INFO:     197.46.110.171:0 - "POST /analyze-data HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [19234]
